# xQTL master workflow template
This notebook provide a master control on the XQTL workflow so it can works on multiple data collection as proposed.
Input:
    A recipe file,each row is a data collection and with the following column:
    
    Theme
        name of dataset, must be different, each uni_study analysis will be performed in a folder named after each, meta analysis will be performed in a folder named as {study1}_{study2}
            
        The column name must contain the # and be the first column
    
    genotype_list
        {Path to file}
    
    molecular_pheno
        {Path to file}
    region_list (list of regions to be analzed)
        {Path to file}
    
    covariate_file
        {Path to file}
    
    factor_analysis_opt
        "APEX" vs "PEER" for factor analysis
    
    LD options:
        "In-sample" LD vs {path to reference panel}
    
    QTL_tool_option
        "APEX" vs "TensorQTL" for QTL association
    
    QTL_analysis_option
        {Int for cis window} vs "trans"
    
    Populations
        The populations from which of the samples was drawn
    
    Conditions:
        The nature of molecular phenotype
    
    ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    
Output:
    ...

## Generation of MWE
This is the code to generate the mwe recipe and LD_recipe on csg cluster

In [ ]:
Recipe_temp = pd.DataFrame( {"Theme" : ["AC","DLPFC","PCC"] ,
                "genotype_list" : ["/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list"],
                "molecular_pheno" : ["/home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt"],
                "region_list" : ["~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region"] ,
                "covariate_file" : ["/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov","/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov","/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov"],
                "factor_analysis_opt" : ["APEX","APEX","APEX"],
                "LD_Recipe": ["~/GIT/ADSPFG-xQTL/MWE/LD_Recipe","~/GIT/ADSPFG-xQTL/MWE/LD_Recipe","~/GIT/ADSPFG-xQTL/MWE/LD_Recipe"],
                "QTL_tool_option" : ["APEX","APEX","APEX"],
                "QTL_analysis_option" : ["cis","cis","cis"],
                "cis_windows" : [500000,500000,5000000],
                "Metal" : ["T","T","F"]}).to_csv("/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example","\t")

        ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    

In [ ]:
pd.DataFrame({"ld_file_prefix" : ["/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename.","/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename."],
               "ld_file_surfix" : [".merged.ld.rds",".merged.ld.rds"]}).to_csv("~/GIT/ADSPFG-xQTL/MWE/LD_Recipe",sep = "\t")





## Example for running the workflow
This will run the workflow from via several submission and save the output to nohup.out

In [ ]:
nohup sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Fine_mapping \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" -J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml -s build &

## Other example workflow:
These command run each of the substep to test them individually

In [ ]:
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb data_proc \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow"

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb QTL \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow"

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Meta \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Fine_mapping \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 




In [ ]:
[global]
## The aforementioned input recipe
parameter: recipe = path
## Overall wd, the file structure of analysis is wd/[steps]/[sub_dir for each steps]
parameter:  wd = path(".")
## Diretory to the excutable
parameter: exe_dir = path("~/GIT/ADSPFG-xQTL/workflow")

parameter: container = '/mnt/mfs/statgen/containers/twas_latest.sif'
parameter: container_lmm = '/mnt/mfs/statgen/containers/lmm.sif'
parameter: container_apex = '/mnt/mfs/statgen/containers/apex.sif'

import pandas as pd
input_inv = pd.read_csv(recipe, sep = "\t")
Metal_theme = input_inv.query("Metal == 'T'")["Theme"].to_list()
Metal_theme_str = "-".join(Metal_theme)
Non_Metal_theme = input_inv.query("Metal != 'T'")["Theme"].to_list()
Non_Metal_theme.append(Metal_theme_str)
Theme_Prefix = "_".join(Non_Metal_theme)

parameter: LD_Recipe  = path(input_inv["LD_Recipe"][0])
input_inv = input_inv.to_dict("records")


## Data Processing


In [ ]:
[data_proc]
input: for_each = "input_inv"
output: f'{wd}/Data_Processing/{_input_inv["Theme"]}.data_proc_output_recipe.tsv'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/Data_Processing/Data_Processing.ipynb Recipe \
        --molecular_pheno_whole $[_input_inv["molecular_pheno"]] \
        --region_list $[_input_inv["region_list"]] \
        --genotype_list $[_input_inv["genotype_list"]] \
        --covariate $[_input_inv["covariate_file"]] \
        --name $[_input_inv["Theme"]] \
        --container_lmm $[container_lmm] \
        --container $[container] \
        --container_apex $[container_apex] \
        --wd $[wd:a]/Data_Processing/ \
        --factor_option $[_input_inv["factor_analysis_opt"]] \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

#### Example:


sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/Data_Processing/Data_Processing.ipynb Recipe \
    --molecular_pheno_whole /home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt \
    --region_list ~/GIT/ADSPFG-xQTL/MWE/mwe_region_long \
    --genotype_list /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list \
    --covariate None \
    --name PCC \
    --container_lmm /mnt/mfs/statgen/containers/lmm.sif \
    --container /mnt/mfs/statgen/containers/twas_latest.sif \
    --container_apex /mnt/mfs/statgen/containers/apex.sif \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Data_Processing/ \
    --factor_option APEX \
    -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

## QTL associations

In [ ]:
[QTL_1]
input: output_from("data_proc"),group_with = "input_inv"
output: f'{wd:a}/QTL_association/{_input_inv["Theme"]}.{_input_inv["QTL_tool_option"]}_{_input_inv["QTL_analysis_option"]}_QTL_recipe.tsv'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/QTL_association/QTL_association.ipynb $[_input_inv["QTL_tool_option"]]_$[_input_inv["QTL_analysis_option"]]_Recipe \
        --recipe $[_input[0]] \
        --container $[container_apex] \
        --window $[_input_inv["cis_windows"]] \
        --name $[_input_inv["Theme"]]  \
        --wd $[wd:a]/QTL_association/ \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

#### Example:

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/QTL_association/QTL_association.ipynb APEX_cis_Recipe \
    --recipe Data_Processing/PCC.data_proc_output_recipe.tsv \
    --container /mnt/mfs/statgen/containers/apex.sif \
    --window 500000 \
    --name PCC  \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/QTL_association/ \
    -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

In [ ]:
[QTL_2]
input: group_by = "all"  
output: f'{_input[0]:d}/sumstat_list'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    import pandas as pd
    input_list = [$[_input:r,]]
    input_inv = $[input_inv]
    chr_temp = [int(x[0].split(".")[-5].replace("chr","")) for x in  pd.read_csv(input_list[0]).values.tolist()]
    sumstat_list = pd.DataFrame({"chr" : chr_temp })
    for x in range(0,len(input_list)):
        sumstat_list = sumstat_list.assign(**{input_inv[x]["Theme"] : pd.read_csv(input_list[x])} )
    sumstat_list.to_csv("$[_output]", index = 0 , sep = "\t")

## Meta Analysis
Input: 
1. A recipe generated from the combination of previouse steps

Output: 
1. Recipe for Prior, Vhat, rds input, resid corr
3. vcf

In [ ]:
[Meta]
input: output_from("QTL_2")
output: f'{wd}/Meta_analysis/Fine_mapping_recipe.txt'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/Meta_Analysis/Meta_Analysis.ipynb Recipe \
        --sumstat_list_path $[_input:a] \
        --region_list $[input_inv[0]["region_list"]] \
        --container $[container] \
        --wd $[wd:a]/Meta_analysis/ \
        --METAL_theme_str $[",".join(Metal_theme)] \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

#### Example
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/Meta_Analysis/Meta_Analysis.ipynb Recipe \
    --sumstat_list_path /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/QTL_association/sumstat_list \
    --region_list ~/GIT/ADSPFG-xQTL/MWE/mwe_region_long \
    --container /mnt/mfs/statgen/containers/twas_latest.sif \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Meta_analysis/ \
    --METAL_theme_str PCC,DLPFC \
    -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

## Fine mapping
Input: 
1. Recipe from previous step
2. a Recipe for LD, need to be made seperately

Output: vcf

In [ ]:
[Fine_mapping]
input: output_from("Meta")
output: f'{wd}/Fine_Mapping/{Theme_Prefix}.unisusie_rss.output_list.txt',
        f'{wd}/Fine_Mapping/{Theme_Prefix}.mvsusie_rss.output_list.txt'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/Fine_Mapping/SumStat/SuSiE_RSS.ipynb Fine_mapping \
        --recipe $[_input:a] \
        --LD_Recipe $[LD_Recipe:a] \
        --container $[container] \
        --wd $[wd:a]/Fine_Mapping/ \
        --Theme_prefix $[Theme_Prefix] # \
        #-J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml &

#### Example
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/Fine_Mapping/SumStat/SuSiE_RSS.ipynb Fine_mapping \
    --recipe /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Meta_analysis/Fine_mapping_recipe.txt \
    --LD_Recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/LD_Recipe \
    --container /mnt/mfs/statgen/containers/twas_latest.sif \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Fine_Mapping/ \
    --Theme_prefix AC_PCC-DLPFC # \
    # -J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml &